# Generación de reportes de análisis de suelos con reportlab

In [11]:
# Importando librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import os.path

In [3]:
# Conectar con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Exploración de datos

In [31]:
# Lectura de archivo Excel con pandas
rutaExcel = '/content/drive/MyDrive/2025-INIA/Python/GenerarReportePDF/Datos/Prueba-Prueba2.csv'
dfDatos = pd.read_csv(rutaExcel, encoding='windows-1252')
dfDatos

,Código de Laboratorio,Matriz Analizada,Fecha de Muestreo (***),Hora de Inicio de Muestreo (h) (***),Condición de la muestra,Código/Identificación de la Muestra por el Cliente (***),pH.,Conductividad Electrica,Materia Orgánica,Fósforo Disponible,Potasio Disponible,Carbonato De Calcio Equivalente,Calcio Intercambiable,Magnesio Intercambiable,Potasio Intercambiable,Sodio Intercambiable,Arena,Arcilla,Limo,Clase Textural
0,SU1149-AQP-24,Suelo,2024-07-18 00:00:00,06:50:00,Conservada,-,7.3,273.0,1.4,58.7,546.90,3.3,102.18,2.00,1.35,4.82,65,9,26,Franco Arenoso
1,SU1150-AQP-24,Suelo,2024-07-18 00:00:00,07:00:00,Conservada,-,7.3,235.0,1.1,24.3,415.03,4.8,41.07,1.87,0.96,4.48,61,11,28,Franco Arenoso
2,SU1151-AQP-24,Suelo,2024-07-18 00:00:00,07:10:00,Conservada,-,7.8,835.0,0.6,43.6,557.12,1.7,107.72,2.30,1.43,14.43,69,11,20,Franco Arenoso
3,SU1152-AQP-24,Suelo,2024-07-18 00:00:00,07:20:00,Conservada,-,7.8,548.0,0.9,26.7,521.57,1.8,136.00,2.20,1.36,10.57,71,11,18,Franco Arenoso
4,SU1153-AQP-24,Suelo,2024-07-18 00:00:00,07:30:00,Conservada,-,7.4,201.0,1.4,36.5,373.55,4.6,37.98,1.77,0.89,5.63,69,11,20,Franco Arenoso
5,SU1154-AQP-24,Suelo,2024-07-18 00:00:00,07:40:00,Conservada,-,7.5,344.0,0.9,64.6,670.25,2.6,87.31,2.07,1.68,5.96,71,9,20,Franco Arenoso
6,SU1155-AQP-24,Suelo,2024-07-18 00:00:00,07:50:00,Conservada,-,7.3,417.3,1.2,508.8,508.83,3.2,74.79,3.18,1.30,5.17,63,9,28,Franco Arenoso
7,SU1156-AQP-24,Suelo,2024-07-18 00:00:00,08:00:00,Conservada,-,7.6,204.0,1.5,399.6,399.63,3.2,45.42,2.85,0.99,3.72,69,13,18,Franco Arenoso
8,SU1157-AQP-24,Suelo,2024-07-18 00:00:00,08:10:00,Conservada,-,7.5,272.0,0.8,488.1,488.13,4.0,102.83,1.54,1.19,2.61,60,8,32,Franco Arenoso
9,SU1158-AQP-24,Suelo,2024-07-18 00:00:00,08:20:00,Conservada,-,7.5,257.0,0.9,537.4,537.36,2.9,36.20,3.41,1.38,3.75,60,10,30,Franco Arenoso


In [32]:
# Mostrar el tipos de dato para cada columna
dfDatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Código de Laboratorio                                     10 non-null     object 
 1   Matriz Analizada                                          10 non-null     object 
 2   Fecha de Muestreo (***)                                   10 non-null     object 
 3   Hora de Inicio de Muestreo (h) (***)                      10 non-null     object 
 4   Condición de la muestra                                   10 non-null     object 
 5   Código/Identificación de la Muestra por el Cliente (***)  10 non-null     object 
 6   pH.                                                       10 non-null     float64
 7   Conductividad Electrica                                   10 non-null     float64
 8   Materia Orgánica       

## 2. Generación de reporte

In [5]:
# Instalar reportlab
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.5 MB/s eta 0:00:00


In [ ]:
# Importando métodos de reportlab
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4, landscape, letter

from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet

from reportlab.lib.units import inch
from reportlab.platypus import (
    BaseDocTemplate, PageTemplate, PageBreak, Frame, FrameBreak, Spacer, Paragraph, HRFlowable, Table, Image
)

from reportlab.pdfgen import canvas

from reportlab.graphics.charts.piecharts import Pie
from reportlab.graphics.shapes import Drawing, Rect

In [10]:
# Visualizando dimensiones de tipo de página A4
A4 # Vertical (595.2755905511812, 841.8897637795277)

width, height = landscape(A4) # Horizontal
width, height # (841.8897637795277, 595.2755905511812)

(841.8897637795277, 595.2755905511812)

In [41]:
# Función para generar reporte
def generarReporte(rutaCSV):

    dfDatos = pd.read_csv(rutaCSV, encoding='windows-1252')

    # Guardar ruta del directorio donde se encuentra el archivo .csv
    head, tail = os.path.split(rutaCSV)

    # Datos de hoja
    width, height = landscape(A4)
    margin = 0.5*inch

    # Generar reporte para cada fila en el archivo .csv
    numFilas = dfDatos.shape[0]
    for i in range(numFilas):
        filaParaReporte = dfDatos.iloc[i]

        # -------------------------------------------
        #              Generar canvas
        # -------------------------------------------

        exportName = os.path.join(head, filaParaReporte['Código de Laboratorio']+'_Reporte.pdf')
        c = canvas.Canvas(exportName, pagesize=landscape(A4))

        # -------------------------------------------
        #              Sección superior
        # -------------------------------------------

        # Imagen-01
        rutaImagen = '/content/drive/MyDrive/2025-INIA/Python/GenerarReportePDF/INIA_logo.png'
        c.drawInlineImage(rutaImagen, margin, height-1.7*margin,
                        width = 1.5*inch*0.75,
                        height= 0.75*inch*0.75)

        # Titulo-01
        c.setFont("Helvetica-Bold", 16)
        posY_titulo = height-1.5*margin
        c.drawCentredString(width/2, posY_titulo,
                            'RESULTADOS DE ENSAYO DE SUELOS AGRÍCOLAS')

        # Linea horizontal 01
        posY_LineaH1 = posY_titulo - 20
        x1, y1, x2, y2 = (margin, posY_LineaH1, width-margin, posY_LineaH1)
        c.line(x1, y1, x2, y2)

        # Dibujar frames (La columna 2 derecha se apoyará en estos datos)
        frame_ancho_completo_y = height - 3 * margin
        frame_ancho_completo_altura = inch

        columna1_x = margin
        columna2_x = width / 2 + margin / 2
        columnas_y = margin
        columnas_ancho = (width - 3 * margin) / 2
        columnas_altura = height - 5 * margin - frame_ancho_completo_altura

        # Tabla-01
        c.setFont("Helvetica", 10)
        valorCodigoLab = filaParaReporte['Código de Laboratorio']
        data = [
            ['Cliente:',             '','','','Fecha de Muestreo: ','','','21/04/2025'],
            ['Cultivo:',    'Arándanos','','','Fecha de Recepción:','','','21/04/2025'],
            ['Código:' , valorCodigoLab,'','','Fecha de Emisión:'  ,'','','21/04/2025']
        ]

        x = 225
        y = posY_LineaH1-10
        ancho_celda = 45
        alto_celda = 17

        num_filas = len(data)
        num_columnas = len(data[0])

        # Dijar Líneas horizontales
        # for i in range(num_filas + 1):
            # c.line(x, y - i * alto_celda,
                # x + num_columnas * ancho_celda, y - i * alto_celda)

        # Dibujar Líneas verticales
        # for j in range(num_columnas + 1):
            # c.line(x + j * ancho_celda, y,
                # x + j * ancho_celda, y - num_filas * alto_celda)

        # Texto en las celdas
        for i, fila in enumerate(data):
            for j, valor in enumerate(fila):
                c.drawString(x + j * ancho_celda + 10,
                            y - (i + 0.8) * alto_celda, str(valor))

        # Línea horizontal 02
        posY_LineaH2 = y - alto_celda*3 - 10
        x1, y1, x2, y2 =  (margin, posY_LineaH2, width-margin, posY_LineaH2)
        c.line(x1, y1, x2, y2)


        # Inicio de dos columnas

        # -------------------------------------------
        #                Columna 1
        # -------------------------------------------
        # c.rect(columna1_x, columnas_y, columnas_ancho, columnas_altura)

        # ------------- Análisis de Textura -----------------
        # Texto-A1
        c.setFont("Helvetica-Bold", 12)
        posY_tituloAnalisis = posY_LineaH2-23
        c.drawString(columna1_x, posY_tituloAnalisis, "Análisis de Textura")

        # Tabla-A1
        c.setFont("Helvetica", 10)
        data1= [
            ['Clase Textural', '', filaParaReporte['Clase Textural'], ''],
            ['Arena',          '', filaParaReporte['Arena']         ,'%'],
            ['Limo',           '', filaParaReporte['Arcilla']       ,'%'],
            ['Arcilla',        '', filaParaReporte['Limo']          ,'%']
        ]
        x = columna1_x
        y = posY_tituloAnalisis - 10

        ancho_celda = 40
        alto_celda = 20

        data = data1
        num_filas = len(data)
        num_columnas = len(data[0])

        # Texto en las celdas
        for i, fila in enumerate(data):
            for j, valor in enumerate(fila):
                c.drawString(x + j * ancho_celda + 10,
                            y - (i + 0.8) * alto_celda, str(valor))

        # Barra-A1 de Análisis de Textura
        drawing = Drawing(100, 200)  # Definimos un tamaño para el dibujo (ancho, alto)

        x_barra = 250  # Posición X de la barra
        y_base = 235  # Posición Y de la base de la barra

        ancho_barra = 40
        altura_total = 90  # Representa el 100% de la altura

        # Colores
        listaColores = [100, 160, 220][::-1]
        # colores = [colors.toColor(f'rgb({colorX},{colorX},{colorX})') for colorX in np.arange(60,240+1, 80)[::-1]] # 3 colores
        colores = [colors.toColor(f'rgb({colorX},{colorX},{colorX})') for colorX in listaColores] # 3 colores
        # np.arange(60, 240+1, 80)[::-1] # 3 colores
        # colores = [
        #     # Colores
        #     # colors.burlywood, colors.rosybrown, colors.chocolate

        #     # Escala de grises
        #     colors.toColor(f'rgb({color1}, {color1}, {color1})'),
        #     colors.toColor(f'rgb({color3}, {color3}, {color3})'),
        #     colors.toColor(f'rgb({color5}, {color5}, {color5})')
        # ]

        porcentajes = [filaParaReporte['Arena'],
                    filaParaReporte['Arcilla'],
                    filaParaReporte['Limo']]

        y_actual = y_base
        for i, porcentaje in enumerate(porcentajes):
            altura_segmento = (porcentaje / 100) * altura_total
            rect = Rect(x_barra, y_actual, ancho_barra, altura_segmento)
            rect.fillColor = colores[i]
            rect.strokeColor = colors.black
            rect.strokeWidth = 1
            drawing.add(rect)
            y_actual += altura_segmento

        drawing.drawOn(c, 50, 100) # Posicionar el dibujo en el lienzo

        # Agregar etiquetas de porcentaje
        c.setFont("Helvetica", 9)
        y_etiqueta = y_base
        for i, porcentaje in enumerate(porcentajes):
            altura_segmento = (porcentaje / 100) * altura_total
            etiqueta = f"{porcentaje}%"
            # Puedes ajustar la posición de la etiqueta según tus necesidades
            c.drawString(x_barra + ancho_barra -10, 100 + y_etiqueta, etiqueta)
            y_etiqueta += altura_segmento

        # ------------- Cationes Cambiables -----------------
        # Texto-A2
        c.setFont("Helvetica-Bold", 12)
        posY_tituloCationes = y - alto_celda*4 - 25
        c.drawString(columna1_x, posY_tituloCationes, "Cationes Cambiables")

        # Tabla-A2
        c.setFont("Helvetica", 10)

        valorCa = filaParaReporte['Calcio Intercambiable']
        valorMg = filaParaReporte['Magnesio Intercambiable']
        valorK  = filaParaReporte['Potasio Intercambiable']
        valorNa = filaParaReporte['Sodio Intercambiable']

        CIC = round(
            valorCa +
            valorMg +
            valorK  +
            valorNa
        , 2)
        data2= [
            [          'Ca', '', valorCa, 'cmol (+)/Kg'],
            [          'Mg', '', valorMg, 'cmol (+)/Kg'],
            [          'K',  '', valorK , 'cmol (+)/Kg'],
            [          'Na', '', valorNa, 'cmol (+)/Kg'],
            ['CIC Efectiva', '',     CIC, 'cmol (+)/Kg']
        ]

        x = columna1_x
        y = posY_tituloCationes - 10

        data = data2
        num_filas = len(data)
        num_columnas = len(data[0])
        alto_celda = 17
        altura_maxima = 25

        # Texto en las celdas
        for i, fila in enumerate(data):
            for j, valor in enumerate(fila):
                if "\n" in str(valor):
                    lineas = str(valor).split("\n")
                    for k, linea in enumerate(lineas):
                        c.drawString(x + j * ancho_celda + 10,
                                    y - i * altura_maxima - (k + 0.2) * 12, linea)
                else:
                    c.drawString(x + j * ancho_celda + 10,
                                y - i * altura_maxima - 0.8 * alto_celda, str(valor))

        # Texto-A3
        c.setFont("Helvetica-Bold", 12)
        posY_tituloDistribucion = y - altura_maxima*5 - 20
        c.drawString(columna1_x, posY_tituloDistribucion, "Distribución de Cationes")

        # Tabla-A3
        c.setFont("Helvetica", 10)

        data3= [
                ['Ca', '', round(valorCa/CIC*100,1),'%'],
                ['Mg', '', round(valorMg/CIC*100,1),'%'],
                ['K',  '', round(valorK /CIC*100,1),'%'],
                ['Na', '', round(valorNa/CIC*100,1),'%'],
            ]

        x = columna1_x
        y = posY_tituloDistribucion - 10

        data = data3
        num_filas = len(data)
        num_columnas = len(data[0])

        alto_celda = 20
        # Texto en las celdas
        for i, fila in enumerate(data):
            for j, valor in enumerate(fila):
                c.drawString(x + j * ancho_celda + 10,
                            y - (i + 0.8) * alto_celda, str(valor))

        # -------------------------------------------
        #                Columna 2
        # -------------------------------------------
        # c.rect(columna2_x, columnas_y, columnas_ancho, columnas_altura)
        # c.drawString(columna2_x + 10, columnas_altura, "Columna 2")

        # ------------- Fertilidad -----------------
        # Texto-B1
        c.setFont("Helvetica-Bold", 12)
        posY_tituloOtrosDatos = posY_tituloAnalisis
        c.drawString(columna2_x - 30, posY_tituloOtrosDatos, "Fertilidad")

        # Tabla-B1
        c.setFont("Helvetica", 10)
        #     < 5.0	Fuertemente ácido
        # 5.1 - 6.5	Moderadamente ácido
        # 6,6 – 7.3	Neutro
        # 7.4 - 8.5	Medianamente alcalino
        #     > 8.5	Alcalino

        def filtrarResultado(rangos, clases, valor):
            if valor < rangos[0]:
                return clases[0]  # Valor menor que 0

            for i in range(len(rangos) - 1):
                if rangos[i] <= valor <= rangos[i + 1]:
                    return clases[i + 1]  # Ajustar índice de clases

            if valor > rangos[-1]:
                return clases[-1]  # Valor mayor o igual a 4

        # pH - Condicionales
        interpretaciones_pH = ['Fuertemente\nácido', 'Moderadamente\nácido',
                            'Neutro', 'Moderadamente\nalcalino', 'Alcalino']
        rangos_pH = [3, 5, 6.5, 7.3, 8.5, 10]
        interpretacion_pH = filtrarResultado(rangos_pH[1:-1],
                                            interpretaciones_pH,
                                            filaParaReporte['pH.'])

        # CE - Condicionales
        interpretaciones_CE = ['No\nsalino', 'Ligeramente\nsalino',
                            'Salino', 'Fuertemente\nsalino', 'Extremadamente\nsalino']
        rangos_CE = [0, 0.25, 0.5, 1, 3, 6]
        interpretacion_CE = filtrarResultado(rangos_CE[1:-1],
                                            interpretaciones_CE,
                                            filaParaReporte['Conductividad Electrica']/100)

        # MO - Condicionales
        interpretaciones_MO = ['Muy\nbajo', 'Bajo',
                            'Medio', 'Alto', 'Muy\nalto']
        rangos_MO = [0, 0.5, 1.5, 3.5, 6, 8]
        interpretacion_MO = filtrarResultado(rangos_MO[1:-1],
                                            interpretaciones_MO,
                                            filaParaReporte['Materia Orgánica '])

        # Fósforo - Condicionales
        interpretaciones_P = ['Bajo', 'Medio', 'Alto']
        rangos_P = [0, 5.5, 11, 16]
        interpretacion_P = filtrarResultado(rangos_P[1:-1],
                                            interpretaciones_P,
                                            filaParaReporte['Fósforo Disponible'])

        # Potasio - Condicionales
        interpretaciones_K = ['Bajo', 'Medio', 'Alto']
        rangos_K = [0, 120, 240, 360]
        interpretacion_K = filtrarResultado(rangos_K[1:-1],
                                            interpretaciones_K,
                                            filaParaReporte['Potasio Disponible'])

        data3= [
                ['pH (1:1)',           '',
                filaParaReporte['pH.'], 'unid.\npH', interpretacion_pH],
                ['C.E. (1:5)',         '',
                filaParaReporte['Conductividad Electrica']/100, 'dS/m', interpretacion_CE],
                ['Materia\nOrgánica',  '',
                filaParaReporte['Materia Orgánica '] , '%', interpretacion_MO],
                ['Fósforo\nDisponible','',
                filaParaReporte['Fósforo Disponible'] , 'mg/kg', interpretacion_P],
                ['Potasio\nDisponible','',
                filaParaReporte['Potasio Disponible'] , 'mg/kg', interpretacion_K]
        ]

        x = columna2_x - 30
        y = posY_tituloOtrosDatos - 12.5

        ancho_celda = 37.5
        alto_celda = 17

        data = data3
        num_filas = len(data)
        num_columnas = len(data[0])

        altura_maxima = 30

        # Texto en las celdas
        for i, fila in enumerate(data):
            for j, valor in enumerate(fila):
                if "\n" in str(valor):
                    lineas = str(valor).split("\n")
                    for k, linea in enumerate(lineas):
                        c.drawString(x + j * ancho_celda + 10,
                                    y - i * altura_maxima - (k + 0.2) * 12, linea)
                else:
                    c.drawString(x + j * ancho_celda + 10,
                                y - i * altura_maxima - 0.8 * alto_celda, str(valor))

        # ------------- Relaciones Catiónicas -----------------
        # Texto-B2
        c.setFont("Helvetica-Bold", 12)
        posY_tituloRelacionesCationicas = (
            posY_tituloOtrosDatos-4*alto_celda-2*altura_maxima-50
        )
        c.drawString(columna2_x - 30,
                    posY_tituloRelacionesCationicas, "Relaciones Catiónicas")

        # Tabla
        c.setFont("Helvetica", 11)
        data4= [
                ['Ca/Mg', '', '', round(valorCa/valorMg, 2), ''],
                [ 'Mg/K', '', '', round(valorMg/valorK , 2), ''],
                [ 'Ca/K', '', '', round(valorCa/valorK , 2), '']]

        x = columna2_x - 30
        y = posY_tituloRelacionesCationicas - 10

        ancho_celda = 40
        alto_celda = 25

        data = data4
        num_filas = len(data)
        num_columnas = len(data[0])

        # Texto en las celdas
        for i, fila in enumerate(data):
            for j, valor in enumerate(fila):
                c.drawString(x + j * ancho_celda + 10,
                            y - (i + 0.8) * alto_celda, str(valor))

        # ---------------------------------------
        # Añadiendo Flowables (Pie y barras) al Canvas
        # ---------------------------------------

        story = []

        # Barras 3 a 4 rangos
        def generar_canvas_barra_rangos(x_inicio, y_barra, rangos, valor_ingresado):
            # Solo afecta a las barras con 3 a 4 rangos

            # Propiedades
            c.setFont("Helvetica", 8)
            # colores = [
            #     # "rgb(242,221,213)", "rgb(232,187,185)", "rgb(194,98,87)",
            #            # "rgb(232,187,185)", "rgb(242,221,213)"
            # '#edf8fb',
            #     # '#b2e2e2',
            #     '#66c2a4','#2ca25f','#006d2c'
            #           ]
            colores = ['#eeeeee','#e0e0e0', '#bdbdbd','#9e9e9e','#757575']

            # x_inicio = 105+1.7*inch
            # y_barra = 700-5.2*inch
            ancho_total = 150
            alto_barra = 10

            # Calcular el ancho de cada unidad en la barra
            # Dividir el ancho total por el rango total
            ancho_rango = ancho_total / (rangos[-1] - rangos[0])

            # Dibujar los rangos de colores en la barra
            x_actual = x_inicio
            for i in range(len(rangos) - 1):

                # Calcular el ancho del rango actual
                ancho_rango_actual = (rangos[i + 1] - rangos[i]) * ancho_rango
                c.setFillColor(colores[i])

                # Dibujar el rectángulo del rango
                c.rect(x_actual, y_barra, ancho_rango_actual, alto_barra, fill=1)

                # Actualizar la posición X para el siguiente rango
                x_actual += ancho_rango_actual

            # Calcular y dibujar la línea horizontal que representa el valor ingresado
            if valor_ingresado >= ancho_total or valor_ingresado > rangos[-1]:
                ancho_valor = ancho_total
            else:
                # Calcular el ancho del valor ingresado
                ancho_valor = (valor_ingresado - rangos[0]) * ancho_rango

            # Calcular la posición Y de la línea
            y_linea = y_barra + alto_barra / 2

            # Propiedades de línea central
            # c.setStrokeColorRGB(255, 0, 0) # Establecer el color de la línea a rojo
            c.setStrokeColorRGB(0, 0, 0) # Establecer el color de la línea a negro

            c.setLineWidth(2.5)              # Establecer el grosor de la línea
            c.line(x_inicio, y_linea, x_inicio + ancho_valor, y_linea)

            # Líneas de tabla
            c.setStrokeColorRGB(0, 0, 0) # Establecer el color de la línea a negro
            c.setLineWidth(1)            # Establecer el grosor de la línea

            # Dibujar las etiquetas de los rangos
            x_etiqueta = x_inicio
            c.setFillColorRGB(0, 0, 0) # Establecer el color de las etiquetas a negro

            len_rangos = len(rangos)
            for i, rango in enumerate(rangos):
                if i == 0:
                    # str(rango) Dibujar la primera etiqueta alineada a la izquierda
                    c.drawString(x_etiqueta-4, y_barra - 10, '')
                else:
                    if i == len_rangos-1:
                        # Dibujar las demás etiquetas alineadas a la derecha
                        c.drawRightString(x_etiqueta + (rango-rangos[0]) * ancho_rango + 4,
                                        y_barra - 10, '')
                    else:
                        # Dibujar las demás etiquetas alineadas a la derecha
                        c.drawRightString(x_etiqueta + (rango-rangos[0]) * ancho_rango + 4,
                                        y_barra - 10, str(rango))

            c.setLineWidth(1) # Establecer el grosor de la línea

        # 5 rangos
        def generar_canvas_barra_rangos_escalada(x_inicio, y_barra,
                                                rangos_reales, valor_ingresado):

            rangos_fijos = [0, 20, 40, 60, 80, 100]
            # colores = ['#edf8fb','#b2e2e2','#66c2a4','#2ca25f','#006d2c']
            colores = ['#eeeeee','#e0e0e0', '#bdbdbd','#9e9e9e','#757575']
            # x_inicio = 100
            # y_barra = 700
            ancho_total = 150
            alto_barra = 10

            ancho_rango = ancho_total / (rangos_fijos[-1] - rangos_fijos[0])

            x_actual = x_inicio
            for i in range(len(rangos_reales) - 1):
                ancho_rango_actual = (rangos_fijos[i + 1] - rangos_fijos[i]) * ancho_rango
                c.setFillColor(colores[i])
                c.rect(x_actual, y_barra, ancho_rango_actual, alto_barra, fill=1)
                x_actual += ancho_rango_actual

            for i in range(len(rangos_reales) - 1):
                # Calcular y dibujar la línea horizontal que representa el valor ingresado
                if valor_ingresado >= ancho_total or valor_ingresado > rangos_reales[-1]:
                    ancho_valor = ancho_total
                if rangos_reales[i] <= valor_ingresado <= rangos_reales[i + 1]:
                    posicion_relativa = (
                        (valor_ingresado - rangos_reales[i]) /
                        (rangos_reales[i + 1] - rangos_reales[i])
                    )
                    ancho_valor = (
                        rangos_fijos[i] + posicion_relativa * (
                            rangos_fijos[i + 1] - rangos_fijos[i])
                    ) * ancho_rango
                    break

            # Propiedades de línea central
            y_linea = y_barra + alto_barra / 2 # Calcular la posición Y de la línea
            # c.setStrokeColorRGB(255, 0, 0) # Establecer el color de la línea a rojo
            c.setStrokeColorRGB(0, 0, 0) # Establecer el color de la línea a negro
            c.setLineWidth(2.5)
            c.line(x_inicio, y_linea, x_inicio + ancho_valor, y_linea)

            # Líneas de tabla color negro
            c.setStrokeColorRGB(0, 0, 0)

            # Mostrar etiquetas bajo las barras
            x_etiqueta = x_inicio
            c.setFillColorRGB(0, 0, 0) # Establecer el color de las etiquetas a negro

            len_rangos = len(rangos_reales)
            for i, rango in enumerate(rangos_fijos):
                if i == 0:
                    # str(rango) Dibujar la primera etiqueta alineada a la izquierda
                    c.drawString(x_etiqueta-4, y_barra - 10, '')
                else:
                    if i == len_rangos-1:
                        # Dibujar las demás etiquetas alineadas a la derecha
                        c.drawRightString(x_etiqueta+(rango-rangos_fijos[0])*ancho_rango+4,
                                        y_barra - 10, '')
                    else:
                        # Dibujar las demás etiquetas alineadas a la derecha
                        c.drawRightString(x_etiqueta+(rango-rangos_fijos[0])*ancho_rango+4,
                                        y_barra - 10, str(rangos_reales[i]))

            c.setLineWidth(1) # Establecer el grosor de líneas de tabla

        # < 5.0	Fuertemente ácido
        # 5.1 - 6.5	Moderadamente ácido
        # 6,6 – 7.3	Neutro
        # 7.4 - 8.5	Medianamente alcalino
        # > 8.5	Alcalino

        # Sección de Barras

        # Lado Izquierdo -----------
        # Barra - CIC
        rangos_ejemplo = [0, 20, 45, 60]

        x_inicio = 115+1.7*inch
        y_barra = posY_tituloCationes - 6.5*20
        valor = CIC
        # story.append(generar_canvas_barra_rangos(x_inicio, y_barra, rangos_ejemplo, valor))

        # Lado Derecho -----------
        c.setFont("Helvetica", 8)
        posX_Barras = columna2_x - 30 + 6*ancho_celda +8
        alto_celda_distancias = 25

        # Barra pH
        #     < 5.0	Fuertemente ácido
        # 5.1 - 6.5	Moderadamente ácido
        # 6,6 – 7.3	Neutro
        # 7.4 - 8.5	Medianamente alcalino
        #     > 8.5	Alcalino

        # Barra - pH
        rangos_ejemplo = [3, 5, 6.5, 7.3, 8.5, 10]
        x_inicio = posX_Barras
        y_barra = posY_tituloOtrosDatos - alto_celda_distancias - 3
        valor = filaParaReporte['pH.']
        story.append(
            generar_canvas_barra_rangos_escalada(x_inicio, y_barra, rangos_ejemplo, valor))

        # Barra - CE
        rangos_ejemplo = [0, 0.25, 0.5, 1, 3, 6]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias - 4
        valor = filaParaReporte['Conductividad Electrica']/100
        story.append(
            generar_canvas_barra_rangos_escalada(x_inicio, y_barra, rangos_ejemplo, valor))

        # Barra - Materia Orgánica
        rangos_ejemplo = [0, 0.5, 1.5, 3.5, 6, 8]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias - 6
        valor = filaParaReporte['Materia Orgánica ']
        story.append(
            generar_canvas_barra_rangos_escalada(x_inicio, y_barra, rangos_ejemplo, valor))

        # c.setFont("Helvetica", 8)
        # Barra - Fósforo Disponible
        rangos_ejemplo = [0, 5.5, 11, 16]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias-4
        valor = filaParaReporte['Fósforo Disponible']
        story.append(
            generar_canvas_barra_rangos(x_inicio, y_barra, rangos_ejemplo, valor))

        # Barra - Potasio Disponible
        rangos_ejemplo = [0, 120, 240, 360]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias-5
        valor = filaParaReporte['Potasio Disponible']
        story.append(
            generar_canvas_barra_rangos(x_inicio, y_barra, rangos_ejemplo, valor))


        # Barras - Relaciones Catiónicas
        # Barra - Ca/mg
        # <1	        Deficiencia de calcio
        # Entre 1 y 2	Bajo nivel del calcio respecto al magnesio
        # Entre 2 y 5	Ideal
        # >5	        Deficiencia de magnesio

        c.setFont("Helvetica", 8)
        rangos_ejemplo = [0, 1, 2, 5, 7]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias - 37.5
        valor = round(valorCa/valorMg, 2)
        story.append(generar_canvas_barra_rangos(x_inicio, y_barra, rangos_ejemplo, valor))

        # Barra - Mg/K
        rangos_ejemplo = [0, 1, 3, 18, 20]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias
        valor = round(valorMg/valorK , 2)
        story.append(generar_canvas_barra_rangos(x_inicio, y_barra, rangos_ejemplo, valor))

        # Barra - Ca/K
        rangos_ejemplo = [0, 30, 60]
        x_inicio = posX_Barras
        y_barra = y_barra - alto_celda_distancias
        valor = round(valorCa/valorK, 2)
        story.append(generar_canvas_barra_rangos(x_inicio, y_barra, rangos_ejemplo, valor))

        # -------------------------------------
        #              Líneas
        # -------------------------------------

        # Línea Vertical Final
        posY = posY_tituloAnalisis+23
        posX_LineaV = width/2-30
        x1, y1, x2, y2 = (posX_LineaV, posY, posX_LineaV, margin)
        c.line(x1, y1, x2, y2)

        # Línea Horizontal Final
        posY = margin
        x1, y1, x2, y2 = (margin, posY, width-margin, posY)
        c.line(x1, y1, x2, y2)

        # -------------------------------------
        #    Añadiendo Flowables al Canvas
        # -------------------------------------

        # from reportlab.lib.styles import getSampleStyleSheet
        styles = getSampleStyleSheet()
        styleN = styles['Normal']
        styleH = styles['Heading1']
        story = []

        # story.append(Paragraph("This is a Heading",styleH))
        # story.append(Paragraph("This is a paragraph in <i>Normal</i> style.", styleN))
        f = Frame(margin, margin, width-2*margin, 9*inch, showBoundary=0)

        # # Gráfico Pie 1: Análisis Textural
        # pie = Pie()
        # pie.x = 220
        # pie.y = -335
        # pie.width = 65
        # pie.height = 65
        # pie.startAngle = 0
        # pie.sideLabels = True
        # pie.slices.fontName = 'Helvetica'
        # pie.slices.labelRadius = 1  # Aumenta la distancia de las etiquetas
        # pie.data = [filaParaReporte['Arena'], filaParaReporte['Arcilla'], filaParaReporte['Limo']]
        # pie.labels = [str(filaParaReporte['Arena'])+'%',
        #               str(filaParaReporte['Arcilla'])+'%',
        #               str(filaParaReporte['Limo'])+'%']
        # # catColor = colors(0.10,0.26,0.46)
        # pie.slices[0].fillColor = colors.toColor('rgb(167,86,44)')
        # pie.slices[1].fillColor = colors.toColor('rgb(219,106,32)')
        # pie.slices[2].fillColor = colors.toColor('rgb(149,14,5)')
        # drawing = Drawing(0, 0)
        # drawing.add(pie)
        # story.append(drawing)


        # Gráfico Pie 2: Distribución de Cationes
        pie = Pie()
        pie.x = 220
        pie.y = -610
        pie.width = 65
        pie.height = 65
        pie.startAngle = 0
        pie.sideLabels = True
        pie.slices.fontName = 'Helvetica'
        pie.slices.labelRadius = 3  # Aumenta la distancia de las etiquetas
        pie.data = [
            filaParaReporte['Calcio Intercambiable']/CIC*100  ,
            filaParaReporte['Magnesio Intercambiable']/CIC*100,
            filaParaReporte['Potasio Intercambiable']/CIC*100 ,
            filaParaReporte['Sodio Intercambiable']/CIC*100
        ]
        pie.labels = [
            str(round(filaParaReporte['Calcio Intercambiable']/CIC*100  ,1))+' %',
            str(round(filaParaReporte['Magnesio Intercambiable']/CIC*100,1))+' %',
            str(round(filaParaReporte['Potasio Intercambiable']/CIC*100 ,1))+' %',
            str(round(filaParaReporte['Sodio Intercambiable']/CIC*100   ,1))+' %'
        ]

        colores = [colors.toColor(f'rgb({colorX},{colorX},{colorX})') for colorX in np.arange(60,240, 40)[::-1]] # 5 colores
        for i, color in enumerate(colores):
            pie.slices[i].fillColor = color
        drawing = Drawing(0, 0)
        drawing.add(pie)

        story.append(drawing)

        f.addFromList(story,c)

        c.save()

In [46]:
# Generamos los reportes para cada fila del archivo .csv el cual se exportará en la misma carpeta
rutaCSV = '/content/drive/MyDrive/2025-INIA/Python/GenerarReportePDF/Datos/Prueba-Prueba2.csv'
generarReporte(rutaCSV)

In [47]:
# Revisamos los reportes generados en la carpeta
os.listdir(os.path.split(rutaCSV)[0])

['Prueba-Prueba2.csv',
 'Prueba-Prueba2.xlsx',
 'SU1149-AQP-24_Reporte.pdf',
 'SU1150-AQP-24_Reporte.pdf',
 'SU1151-AQP-24_Reporte.pdf',
 'SU1152-AQP-24_Reporte.pdf',
 'SU1153-AQP-24_Reporte.pdf',
 'SU1154-AQP-24_Reporte.pdf',
 'SU1155-AQP-24_Reporte.pdf',
 'SU1156-AQP-24_Reporte.pdf',
 'SU1157-AQP-24_Reporte.pdf',
 'SU1158-AQP-24_Reporte.pdf']